# **Hubungkan ke google drive**

In [1]:
# Periksa apakah NVIDIA GPU diaktifkan
!nvidia-smi

Fri Aug 28 09:57:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
'10 Webinar Inixindo Surabaya Rekayasa Perangkat Lunak Berbasis Agile.mp3'
'14 Optimalisasi It Di Perusahaan.mp3'
'1904.08378 (1).pdf'
'2003.05997 (1).pdf'
'20200529_Common Mistakes When Applying for Scholarships.pdf'
'2020 06 16_Rektor IPB_Webinar IPMI.pdf'
'26 Rakha.pdf'
 4_5968789044220396029.pdf
 463.pdf
'ABD-Diskominfo PROBOLINGGO_4800364977 - 4800371630.gdoc'
'ABD-Diskominfo PROBOLINGGO_4800364977 - 4800371630.pdf'
'[Adrian_Rosebrock]_Deep_Learning_for_

## **1) Cloning repo the Darknet**



In [3]:
# Darknet adalah framework untuk training yolo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14263, done.
remote: Total 14263 (delta 0), reused 0 (delta 0), pack-reused 14263
Receiving objects: 100% (14263/14263), 12.85 MiB | 15.91 MiB/s, done.
Resolving deltas: 100% (9722/9722), done.


## **2) Mengcompile Darknet menggunakan Nvidia GPU**


In [4]:
# compile Darknet yang telah didownload agar bekerja menggunakan GPU &
# ubah makefile agar GPU dan OPENCV diaktifkan
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

## **3) konfigurasi Darknet network untuk training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Konfigurasi untuk 1 kelas
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Buat folder di google drive sehingga kita bisa menyimpan weight/bobot di sana
!mkdir "/mydrive/yolov3"

In [ ]:
!echo "face_aja" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-06-10 08:21:51--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  62.8KB/s    in 33m 27s 

2020-06-10 08:55:19 (79.1 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

Gambar harus berada bentuk arsip zip dengan nama "images.zip" dan harus berada di dalam folder "yolov3" di Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

unzip:  cannot find or open /mydrive/yolov3/images.zip, /mydrive/yolov3/images.zip.zip or /mydrive/yolov3/images.zip.ZIP.


In [ ]:
# convert label dari class index pada semua file anotasi *.txt. Karena hanya 1 kelas, nilainya adalah 0.
# Jika nilainya beda, harus diganti.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.483146 0.378440 0.415730 0.500000

['0', '0.483146', '0.378440', '0.415730', '0.500000']
0 data/obj/output415.txt
0 0.483146 0.378440 0.415730 0.500000
0 0.544944 0.600917 0.460674 0.458716

['0', '0.544944', '0.600917', '0.460674', '0.458716']
1 data/obj/000257.txt
0 0.544944 0.600917 0.460674 0.458716
0 0.617978 0.582569 0.505618 0.587156

['0', '0.617978', '0.582569', '0.505618', '0.587156']
2 data/obj/000036.txt
0 0.617978 0.582569 0.505618 0.587156
0 0.522472 0.628440 0.471910 0.495413

['0', '0.522472', '0.628440', '0.471910', '0.495413']
3 data/obj/000299.txt
0 0.522472 0.628440 0.471910 0.495413
0 0.508427 0.621560 0.556180 0.490826

['0', '0.508427', '0.621560', '0.556180', '0.490826']
4 data/obj/output3.txt
0 0.508427 0.621560 0.556180 0.490826
0 0.494382 0.371560 0.449438 0.339450

['0', '0.494382', '0.371560', '0.449438', '0.339450']
5 data/obj/output206.txt
0 0.494382 0.371560 0.449438 0.339450
0 0.514045 0.385321 0.432584 0.403670

['0', '0.514045', '0.385321', '0.432

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/000243.jpg', 'data/obj/000042.jpg', 'data/obj/000108.jpg', 'data/obj/000172.jpg', 'data/obj/output191.jpg', 'data/obj/output245.jpg', 'data/obj/output324.jpg', 'data/obj/output220.jpg', 'data/obj/output442.jpg', 'data/obj/output9.jpg', 'data/obj/output375.jpg', 'data/obj/output107.jpg', 'data/obj/000396.jpg', 'data/obj/000086.jpg', 'data/obj/000139.jpg', 'data/obj/output696.jpg', 'data/obj/000077.jpg', 'data/obj/output78.jpg', 'data/obj/output685.jpg', 'data/obj/output419.jpg', 'data/obj/000304.jpg', 'data/obj/000374.jpg', 'data/obj/output50.jpg', 'data/obj/000322.jpg', 'data/obj/output144.jpg', 'data/obj/output333.jpg', 'data/obj/output257.jpg', 'data/obj/output403.jpg', 'data/obj/output689.jpg', 'data/obj/output91.jpg', 'data/obj/000072.jpg', 'data/obj/output354.jpg', 'data/obj/output400.jpg', 'data/obj/000192.jpg', 'data/obj/000062.jpg', 'data/obj/000180.jpg', 'data/obj/output384.jpg', 'data/obj/000305.jpg', 'data/obj/000022.jpg', 'data/obj/000456.jpg', 'data/obj/000365.j

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.749284, GIOU: 0.749072), Class: 0.997644, Obj: 0.533802, No Obj: 0.002290, .5R: 1.000000, .75R: 0.500000, count: 4, class_loss = 0.347888, iou_loss = 0.239259, total_loss = 0.587146 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000020, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000002, iou_loss = 0.000000, total_loss = 0.000002 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000007, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.678646, GIOU: 0.678457), Class: 0.995988, Obj: 0.515972, No Obj: 0.001961, .5R: 0.750000, .75R: 0.500000, count: 4, class_loss = 0.472753, iou_loss = 0.297682, total_